In [2]:
List<String> myList =
    Arrays.asList("a1", "a2", "b1", "c2", "c1");

myList
    .stream()
    .filter(s -> s.startsWith("c"))
    .map(String::toUpperCase)
    .sorted()
    .forEach(System.out::println);

C1
C2


In [14]:
import java.util.stream.IntStream;
import java.util.List;
import java.util.stream.Collectors;
import java.util.stream.Stream;

IntStream.range(1, 100)
.mapToObj(n -> (n % 5 == 0 && n % 3 == 0) ? "FizzBuzz"
  : (n % 5 == 0 ? "Fizz"
    : n % 3 == 0 ? "Buzz"
      : Integer.toString (n)) )
.forEach(System.out::println);

1
2
Buzz
4
Fizz
Buzz
7
8
Buzz
Fizz
11
Buzz
13
14
FizzBuzz
16
17
Buzz
19
Fizz
Buzz
22
23
Buzz
Fizz
26
Buzz
28
29
FizzBuzz
31
32
Buzz
34
Fizz
Buzz
37
38
Buzz
Fizz
41
Buzz
43
44
FizzBuzz
46
47
Buzz
49
Fizz
Buzz
52
53
Buzz
Fizz
56
Buzz
58
59
FizzBuzz
61
62
Buzz
64
Fizz
Buzz
67
68
Buzz
Fizz
71
Buzz
73
74
FizzBuzz
76
77
Buzz
79
Fizz
Buzz
82
83
Buzz
Fizz
86
Buzz
88
89
FizzBuzz
91
92
Buzz
94
Fizz
Buzz
97
98
Buzz
